In [7]:
import os
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import seaborn as sns

# Список директорий с наборами данных
dataset_dirs = ['DataSet_b_15', 'DataSet_b_20', 'DataSet_b_25', 'DataSet_s_b_15', 'DataSet_s_b_20', 'DataSet_s_b_25']  # Список папок с разными наборами данных

# Функция для разделения данных на тренировочную, валидационную и тестовую выборки
def split_data(dataset_dir, classes, validation_split=0.2, test_split=0.1):
    train_dir = os.path.join(dataset_dir, 'train')
    validation_dir = os.path.join(dataset_dir, 'validation')
    test_dir = os.path.join(dataset_dir, 'test')
    
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(validation_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)
    
    for class_name in classes:
        class_dir = os.path.join(dataset_dir, class_name)
        
        if not os.path.exists(class_dir):
            print(f"Ошибка: директория {class_dir} не найдена.")
            continue
        
        images = os.listdir(class_dir)
        
        if not images:
            print(f"Ошибка: директория {class_dir} пуста.")
            continue
        
        train_images, test_images = train_test_split(images, test_size=test_split, random_state=42)
        train_images, val_images = train_test_split(train_images, test_size=validation_split, random_state=42)
        
        train_class_dir = os.path.join(train_dir, class_name)
        validation_class_dir = os.path.join(validation_dir, class_name)
        test_class_dir = os.path.join(test_dir, class_name)
        
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(validation_class_dir, exist_ok=True)
        os.makedirs(test_class_dir, exist_ok=True)
        
        for img in train_images:
            shutil.copy(os.path.join(class_dir, img), os.path.join(train_class_dir, img))
        for img in val_images:
            shutil.copy(os.path.join(class_dir, img), os.path.join(validation_class_dir, img))
        for img in test_images:
            shutil.copy(os.path.join(class_dir, img), os.path.join(test_class_dir, img))

# Основной процесс обучения
for dataset_dir in dataset_dirs:
    print(f"Обработка датасета: {dataset_dir}")
    classes = ['class1', 'class2', 'class3', 'class4', 'class5', 'class6', 'class7', 'class8']
    split_data(dataset_dir, classes)
    
    train_dir = os.path.join(dataset_dir, 'train')
    validation_dir = os.path.join(dataset_dir, 'validation')
    test_dir = os.path.join(dataset_dir, 'test')
    
    batch_size = 32
    image_size = 128
    
    train_datagen = ImageDataGenerator(rescale=1./255)
    val_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(train_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical')
    validation_generator = val_datagen.flow_from_directory(validation_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical')
    test_generator = test_datagen.flow_from_directory(test_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical', shuffle=False)
    
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(image_size, image_size, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(len(classes), activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    history = model.fit(train_generator, validation_data=validation_generator, epochs=10)
    
    model.save(f'{dataset_dir}_model.h5')
    
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{dataset_dir} - Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig(f'{dataset_dir}_accuracy.png')
    plt.close()
    
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{dataset_dir} - Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'{dataset_dir}_loss.png')
    plt.close()
    
    test_loss, test_accuracy = model.evaluate(test_generator)
    print(f"{dataset_dir} - Test Accuracy: {test_accuracy}")
    
    y_pred = model.predict(test_generator)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = test_generator.classes
    
    cm = confusion_matrix(y_true, y_pred_classes)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title(f'{dataset_dir} - Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(f'{dataset_dir}_confusion_matrix.png')
    plt.close()
    
    with open(f'{dataset_dir}_classification_report.txt', 'w') as f:
        f.write(classification_report(y_true, y_pred_classes, target_names=classes))
    
    print(f"Завершена обработка {dataset_dir}\n")


Обработка датасета: DataSet_b_15
Found 10480 images belonging to 8 classes.
Found 2624 images belonging to 8 classes.
Found 1456 images belonging to 8 classes.
Epoch 1/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 116s 351ms/step - accuracy: 0.4406 - loss: 1.5309 - val_accuracy: 0.9120 - val_loss: 0.2844
Epoch 2/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 44s 133ms/step - accuracy: 0.9237 - loss: 0.2638 - val_accuracy: 0.9546 - val_loss: 0.1584
Epoch 3/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 38s 116ms/step - accuracy: 0.9601 - loss: 0.1319 - val_accuracy: 0.9619 - val_loss: 0.1243
Epoch 4/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 43s 130ms/step - accuracy: 0.9784 - loss: 0.0770 - val_accuracy: 0.9802 - val_loss: 0.0684
Epoch 5/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 44s 134ms/step - accuracy: 0.9878 - loss: 0.0415 - val_accuracy: 0.9737 - val_loss: 0.0850
Epoch 6/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 42s 128ms/step - accuracy: 0.9923 - loss: 0.0263 - val_accuracy: 0.9817 - val_loss: 0.0589
Epoch 7/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 38s 116ms/ste

46/46 ━━━━━━━━━━━━━━━━━━━━ 9s 194ms/step - accuracy: 0.9669 - loss: 0.1039
DataSet_b_15 - Test Accuracy: 0.9828296899795532
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step
Завершена обработка DataSet_b_15

Обработка датасета: DataSet_b_20
Found 10480 images belonging to 8 classes.
Found 2624 images belonging to 8 classes.
Found 1456 images belonging to 8 classes.


C:\Users\dmitr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\dmitr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 110s 331ms/step - accuracy: 0.4704 - loss: 1.4395 - val_accuracy: 0.9139 - val_loss: 0.2867
Epoch 2/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 39s 118ms/step - accuracy: 0.9248 - loss: 0.2437 - val_accuracy: 0.9459 - val_loss: 0.1730
Epoch 3/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 38s 117ms/step - accuracy: 0.9591 - loss: 0.1369 - val_accuracy: 0.9672 - val_loss: 0.1026
Epoch 4/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 37s 114ms/step - accuracy: 0.9713 - loss: 0.0865 - val_accuracy: 0.9756 - val_loss: 0.0852
Epoch 5/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 39s 119ms/step - accuracy: 0.9864 - loss: 0.0474 - val_accuracy: 0.9874 - val_loss: 0.0475
Epoch 6/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 38s 116ms/step - accuracy: 0.9923 - loss: 0.0294 - val_accuracy: 0.9623 - val_loss: 0.1022
Epoch 7/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 39s 119ms/step - accuracy: 0.9906 - loss: 0.0285 - val_accuracy: 0.9916 - val_loss: 0.0378
Epoch 8/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 38s 116ms/step - accuracy: 0.9981 - loss: 

46/46 ━━━━━━━━━━━━━━━━━━━━ 8s 186ms/step - accuracy: 0.9808 - loss: 0.0577
DataSet_b_20 - Test Accuracy: 0.9855769276618958
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step
Завершена обработка DataSet_b_20

Обработка датасета: DataSet_b_25
Found 10480 images belonging to 8 classes.
Found 2624 images belonging to 8 classes.
Found 1456 images belonging to 8 classes.


...

Epoch 1/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 104s 315ms/step - accuracy: 0.2821 - loss: 1.8792 - val_accuracy: 0.8598 - val_loss: 0.4320
Epoch 2/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 38s 116ms/step - accuracy: 0.8557 - loss: 0.4366 - val_accuracy: 0.9139 - val_loss: 0.2691
Epoch 3/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 35s 106ms/step - accuracy: 0.9136 - loss: 0.2611 - val_accuracy: 0.9451 - val_loss: 0.1761
Epoch 4/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 35s 106ms/step - accuracy: 0.9360 - loss: 0.2010 - val_accuracy: 0.9348 - val_loss: 0.2004
Epoch 5/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 35s 105ms/step - accuracy: 0.9487 - loss: 0.1593 - val_accuracy: 0.9493 - val_loss: 0.1580
Epoch 6/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 35s 105ms/step - accuracy: 0.9604 - loss: 0.1273 - val_accuracy: 0.9630 - val_loss: 0.1134
Epoch 7/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 35s 106ms/step - accuracy: 0.9673 - loss: 0.0954 - val_accuracy: 0.9512 - val_loss: 0.1262
Epoch 8/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 35s 105ms/step - accuracy: 0.9747 - loss: 

46/46 ━━━━━━━━━━━━━━━━━━━━ 8s 178ms/step - accuracy: 0.9584 - loss: 0.1475
DataSet_s_b_20 - Test Accuracy: 0.9711538553237915
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step
Завершена обработка DataSet_s_b_20

Обработка датасета: DataSet_s_b_25
Found 10480 images belonging to 8 classes.
Found 2624 images belonging to 8 classes.
Found 1456 images belonging to 8 classes.


C:\Users\dmitr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\dmitr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 100s 302ms/step - accuracy: 0.3058 - loss: 1.8137 - val_accuracy: 0.8563 - val_loss: 0.4659
Epoch 2/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 40s 121ms/step - accuracy: 0.8643 - loss: 0.4495 - val_accuracy: 0.8899 - val_loss: 0.3275
Epoch 3/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 40s 121ms/step - accuracy: 0.9200 - loss: 0.2688 - val_accuracy: 0.8887 - val_loss: 0.3457
Epoch 4/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 36s 110ms/step - accuracy: 0.9277 - loss: 0.2278 - val_accuracy: 0.9066 - val_loss: 0.2699
Epoch 5/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 36s 110ms/step - accuracy: 0.9401 - loss: 0.1814 - val_accuracy: 0.9356 - val_loss: 0.2022
Epoch 6/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 37s 113ms/step - accuracy: 0.9510 - loss: 0.1528 - val_accuracy: 0.9386 - val_loss: 0.1771
Epoch 7/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 36s 109ms/step - accuracy: 0.9510 - loss: 0.1467 - val_accuracy: 0.9505 - val_loss: 0.1465
Epoch 8/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 38s 117ms/step - accuracy: 0.9617 - loss: 

46/46 ━━━━━━━━━━━━━━━━━━━━ 9s 200ms/step - accuracy: 0.9506 - loss: 0.1379
DataSet_s_b_25 - Test Accuracy: 0.9375
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step
Завершена обработка DataSet_s_b_25

